In [1]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Data Wrangling
import numpy as np
import pandas as pd
from pathlib import Path
import glob
import os
import flammkuchen as fl


## Setup paths

In [3]:
# save_data_path = Path(r"\\portulab.synology.me\data\Kata\Processed_Data\tracking_assessment")

# save_data_path = Path(r'\\portulab.synology.me\data\Kata\testdata\Processed_Data')
# save_data_path

In [24]:
# master_path = Path(r"\\portulab.synology.me\data\Kata\Data\230307_visstim_2D")
# master_path = Path(r"\\portulab.synology.me\data\Kata\Data\22042024_visstim_2D_round")
# master_path = Path(r"\\portulab.synology.me\data\Kata\Data\22042024_visstim_2D_2")
# master_path = Path(r"\\portulab.synology.me\data\Kata\Data\13052024_visstim_2D_round")
master_path = Path(r"\\portulab.synology.me\data\Kata\Data\14052024_visstim_2D_round")

# master_path = Path(r'\\portulab.synology.me\data\Kata\testdata\Raw_Data')

fish_paths = list(master_path.glob('*f[0-9]*'))
fish_paths

[WindowsPath('//portulab.synology.me/data/Kata/Data/14052024_visstim_2D_round/240514_f0'),
 WindowsPath('//portulab.synology.me/data/Kata/Data/14052024_visstim_2D_round/240514_f1'),
 WindowsPath('//portulab.synology.me/data/Kata/Data/14052024_visstim_2D_round/240514_f2'),
 WindowsPath('//portulab.synology.me/data/Kata/Data/14052024_visstim_2D_round/240514_f3'),
 WindowsPath('//portulab.synology.me/data/Kata/Data/14052024_visstim_2D_round/240514_f4'),
 WindowsPath('//portulab.synology.me/data/Kata/Data/14052024_visstim_2D_round/240514_f5'),
 WindowsPath('//portulab.synology.me/data/Kata/Data/14052024_visstim_2D_round/240514_f6'),
 WindowsPath('//portulab.synology.me/data/Kata/Data/14052024_visstim_2D_round/240514_f7'),
 WindowsPath('//portulab.synology.me/data/Kata/Data/14052024_visstim_2D_round/240514_f8'),
 WindowsPath('//portulab.synology.me/data/Kata/Data/14052024_visstim_2D_round/240514_f9')]

In [25]:
fish = 1
fish_path = fish_paths[fish]
exp_name = Path(fish_path).parts[-2] 
# exp_name = 'testfish'
exp_name

'14052024_visstim_2D_round'

In [26]:
# out_path = Path(r"\\portulab.synology.me\data\Kata\Processed_Data\230307_visstim_2D_")
# out_path = Path(r"\\portulab.synology.me\data\Kata\Processed_Data\22042024_visstim_2D_round_")
# out_path = Path(r"\\portulab.synology.me\data\Kata\Processed_Data\22042024_visstim_2D_2_")
# out_path = Path(r"\\portulab.synology.me\data\Kata\Processed_Data\13052024_visstim_2D_round_")
out_path = Path(r"\\portulab.synology.me\data\Kata\Processed_Data\14052024_visstim_2D_round_")

# out_path = Path(r'\\portulab.synology.me\data\Kata\testdata\Processed_Data')


save_data_path = out_path

## Calculate IBI for all fish

In [27]:
thr =0.95
ibis_ = []

for path in fish_paths:
    fish_id = path.name
    print (fish_id)
    try:
        dlc_path_ = Path(out_path/ '{}_DLC_mod.csv'.format(fish_id)) 
        df_dlc = pd.read_csv(dlc_path_, header=[0,1])
        mask = df_dlc.body.likelihood >=thr
        
        # get bout starts
        data_path = list(out_path.glob('*{}_megabouts_res.h5*'.format(fish_id)))[0]
        data= fl.load(data_path)
        data = data['data']
        
        # get bout starts
        data_path_2 = list(out_path.glob('*{}_bout_data.h5*'.format(fish_id)))[0]
        data_2= fl.load(data_path_2)
    
        on = data['segments_on']
        off =data['segments_off']
        print (on.shape[0])
    
        ibi = []
        for i in range(on.shape[0]-1):
            ibi.append(df_dlc.t.iloc[on[i+1]][0] - df_dlc.t.iloc[off[i]][0])
    
        ibi=np.asarray(ibi)
        ## do calculations and save  
        ### just distances in active tracking episodes
        exp_time = np.arange(0, df_dlc.shape[0], 1)
        start_frames = data_2['bout_times'][:, 0]  # fetch all start frames from bout_times
        masked_exp_time = exp_time[mask==True]
        
        # Now, check which start frames are in masked_exp_time
        start_frames_in_mask = np.isin(start_frames, masked_exp_time)
        print(ibi.shape, start_frames_in_mask.shape)
    
        if ibi.shape[0] != start_frames_in_mask.shape[0]:
            ibi_tracking = ibi[start_frames_in_mask[:-1]]
        else: 
            ibi_tracking = ibi[start_frames_in_mask]
        fl.save(out_path/ '{}_ibi.h5'.format(fish_id), ibi_tracking)
        ibis_.append(ibi_tracking)
    except:
        pass

fl.save(save_data_path/ '{}_ibi_pooled.h5'.format(exp_name), ibis_)


240514_f0
240514_f1
240514_f2
240514_f3
240514_f4
240514_f5
240514_f6
240514_f7
240514_f8
240514_f9
